In [1]:
import pandas as pd
import os

In [10]:
def get_list_source_target(lang='EN'):
    list_dir = os.listdir('../Small/')
    list_dir_en_source = [x for x in list_dir if (lang+'-') in x]
    list_dir_en_target = [x for x in list_dir if ('-'+lang) in x]
    return list_dir_en_source, list_dir_en_target

In [64]:
langs = ['PT', 'AN', 'RO', 'SC', 'FR', 'CA', 'IT', 'EN', 'EU', 'EO', 'ES', 'OC', 'GL']
def get_lang_src_target(path):
    src_target = path.split('_')[0].split('Set')[1].split('-')
    return src_target
def get_list_source_target_infe(lang='EN'):
    
    list_dir = os.listdir('../Small/')
    list_dir = [x for x in list_dir if '.csv' in x]
    list_dir_en_source = [x for x in list_dir if get_lang_src_target(x)[0]==lang and get_lang_src_target(x)[1] in langs]
    list_dir_en_target = [x for x in list_dir if get_lang_src_target(x)[1]==lang and get_lang_src_target(x)[0] in langs]
    return list_dir_en_source, list_dir_en_target

In [65]:
get_list_source_target_infe()

(['TransSetEN-GL_small.csv',
  'TransSetEN-ES_small.csv',
  'TransSetEN-CA_small.csv'],
 ['TransSetEU-EN_small.csv', 'TransSetEO-EN_small.csv'])

In [13]:
def get_data_frame_lang(lang='EN'):
    df_en= pd.DataFrame(columns=['source', 'POS'])
    source, target = get_list_source_target(lang)
    for path_source in source:
      path_source = '../Small/' + path_source 
      df = pd.read_csv(path_source, delimiter=',', error_bad_lines=False)
      df_form_source = df[['source', 'POS']]
      df_en = pd.concat([df_en, df_form_source], ignore_index=True)
    for path_source in target:
      path_source =  '../Small/' + path_source 
      df = pd.read_csv(path_source, delimiter=',', error_bad_lines=False)
      df_form_source = df[['target', 'POS']]
      df_form_source.columns = ['source', 'POS']
      df_en = pd.concat([df_en, df_form_source], ignore_index=True)
    return df_en

In [24]:
for lang in ['EN', 'FR', 'PT']:
    df = get_data_frame_lang(lang)
    df = df.drop_duplicates(ignore_index=True)
    df = df.sort_values(by=['POS'], ignore_index=True)
    path = '../Small/'+lang+'_vocab/'
    
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
    
    df.to_csv(path+'all.csv', index=False)
    max_rows = 4500
    dataframes = []
    while len(df) > max_rows:
        top = df[:max_rows]
        dataframes.append(top)
        df = df[max_rows:]
    else:
        dataframes.append(df)
    for _, frame in enumerate(dataframes):
        frame.to_csv(path+str(_)+'.csv', index=False)

Successfully created the directory ../Small/EN_vocab/ 
Successfully created the directory ../Small/FR_vocab/ 
Successfully created the directory ../Small/PT_vocab/ 


In [28]:
all_en = pd.read_csv('../Small/EN_vocab/all.csv')
all_fr = pd.read_csv('../Small/Fr_vocab/all.csv')
all_pt = pd.read_csv('../Small/PT_vocab/all.csv')

In [29]:
print('len en: {},len fr: {}, len pt: {}'.format(len(all_en), len(all_fr), len(all_pt)))

len en: 93466,len fr: 87146, len pt: 51035


In [31]:
check_en_fr = pd.merge(all_en, all_fr, on=['source', 'POS'], how='inner')

In [47]:
all_en.merge(all_fr,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']]

,source,POS
4,adolescent,adjective
9,militant,adjective
17,compact,adjective
28,adjectival,adjective
29,changeable,adjective
...,...,...
89135,flatter,verb
89998,layer,verb
90286,solder,verb
92090,twitter,verb


In [55]:
#get difference between en_fr
all_en.merge(all_fr,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_en_fr.csv', index=False)
all_en.merge(all_fr,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']].to_csv('../Small/Dictionary_Same/same_en_fr.csv', index=False)

In [61]:
#get difference between en_pt
all_en.merge(all_pt,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_en_pt.csv', index=False)
#get difference same en_pt
all_en.merge(all_pt,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']].to_csv('../Small/Dictionary_Same/same_en_pt.csv', index=False)

In [57]:
#get difference between fr_en
all_fr.merge(all_en,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_fr_en.csv', index=False)
all_fr.merge(all_en,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']].to_csv('../Small/Dictionary_Same/same_fr_en.csv', index=False)

In [58]:
#get difference between fr_pt
all_fr.merge(all_pt,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_fr_pt.csv', index=False)
#get difference same fr_pt
all_fr.merge(all_pt,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']].to_csv('../Small/Dictionary_Same/same_fr_pt.csv', index=False)

In [59]:
#get difference between pt_en
all_pt.merge(all_en,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_pt_en.csv', index=False)
#get difference same pt_en
all_pt.merge(all_en,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']].to_csv('../Small/Dictionary_Same/same_pt_en.csv', index=False)

In [60]:
#get difference between pt_fr
all_pt.merge(all_fr,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_pt_fr.csv', index=False)
#get difference same pt_fr
all_pt.merge(all_fr,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']].to_csv('../Small/Dictionary_Same/same_pt_fr.csv', index=False)

In [66]:
def get_data_frame_lang_infe(lang='EN'):
    df_en= pd.DataFrame(columns=['source', 'POS'])
    source, target = get_list_source_target_infe(lang)
    for path_source in source:
      path_source = '../Small/' + path_source 
      df = pd.read_csv(path_source, delimiter=',', error_bad_lines=False)
      df_form_source = df[['source', 'POS']]
      df_en = pd.concat([df_en, df_form_source], ignore_index=True)
    for path_source in target:
      path_source =  '../Small/' + path_source 
      df = pd.read_csv(path_source, delimiter=',', error_bad_lines=False)
      df_form_source = df[['target', 'POS']]
      df_form_source.columns = ['source', 'POS']
      df_en = pd.concat([df_en, df_form_source], ignore_index=True)
    return df_en

In [69]:
df_en_infe = get_data_frame_lang_infe().drop_duplicates(ignore_index=True)
df_fr_infe = get_data_frame_lang_infe('FR').drop_duplicates(ignore_index=True)
df_pt_infe = get_data_frame_lang_infe('PT').drop_duplicates(ignore_index=True)

In [73]:
df_en_infe.merge(df_fr_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_en_fr.csv', index=False)
df_en_infe.merge(df_pt_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_en_pt.csv', index=False)

df_fr_infe.merge(df_en_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_fr_en.csv', index=False)
df_fr_infe.merge(df_pt_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_fr_pt.csv', index=False)

df_pt_infe.merge(df_en_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_pt_en.csv', index=False)
df_pt_infe.merge(df_fr_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_pt_fr.csv', index=False)